---
### **<p style="text-align: center; text-decoration: underline;">Bayesian Machine Learning Project</p>**
# **<p style="text-align: center;">Bayesian Learning via Stochastic Gradient Langevin Dynamics</p>**
---

> Realized by: *Zakaria Boulkhir* & *Omar Iken*.

> Master 2, Data Science, Lille University.

---

### ■ __Overview__
This [project](https://github.com/rbardenet/bml-course/blob/m2-lille/projects/papers.pdf) is part of the lecture on *Bayesian Machine Learning* tought by [Rémi BARDENET](https://rbardenet.github.io/). The idea is to pick a paper from a list of given papers, and read it with a critical mind. For instance, we will:
- (1) explain the contents of the paper
- (2) emphasize the strong and weak points of the paper
- (3) apply it to real data of our choice when applicable.

So this notebook concerns the implementation part of the project.

### ■ __Article__
M. Welling and Y. W. Teh. Bayesian learning via stochastic gradient Langevin
dynamics. In Proceedings of the 28th international conference on machine learning
(ICML-11), pages 681–688, 2011.

The main article is available [here](http://people.ee.duke.edu/~lcarin/398_icmlpaper.pdf).

### ■ **<a name="content">Contents</a>**

- [I. Dataset](#dataset)

- [2. Implementation](#imp)

    - [2.1. Preliminaries]()
    
    - [2.2. Stochastic Gradient Langevin Dynamics (SGLD)]()
    
    - [2.3. Posterior Sampling]()
    
    
- [3. Evaluation](#eval)

    - [3.1. Mixture of Gaussians]()
    
    - [3.2. Logistic Regression]()
    
    - [3.3. Independent Components Analysis]()

### ■ **Libraries**
Let's start by uploading all the libraries needed for this project.

In [1]:
## numpy to handle arrays & matices
import numpy as np

## matplotlib & Seaborn to plot figures
import matplotlib.pyplot as plt
import seaborn as sns

## pandas to handle dataframes
import pandas as pd
from scipy import stats

#-----------< Setting >------------#
## set plots text font size & style
sns.set(font_scale=1.2, style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

### ■ **<a name="dataset">1. Dataset</a>** [(&#8593;)](#content)
In this first section, we will load the dataset that we will use during this project. Then we will do some preprocessing and exploration.

#### **1.2. Artificial Data**
During this project, we will be working with both artificial and real data. The artificial data is generated as data-cases iid in six channels. Three channels had high kurtosis distributions while three others where normally distributed. The total number of generated samples is 1000.

In [14]:
def generate_art_data(n_samples):
    """generate artificial data"""
    
    normal_samples = np.random.normal(size=(n_samples, n_samples, 3))
    
    kurtosis_samples = stats.kurtosis(np.random.normal(size=(n_samples, n_samples, n_samples, 3)), fisher=True)

    data = np.concatenate((normal_samples, kurtosis_samples), axis=-1)
    
    return data

In [16]:
data = generate_art_data(100)

data.shape

(100, 100, 6)

#### **1.2. MEG Data**

### ■ **<a name="imp">2. Implementation</a>** [(&#8593;)](#content)
This second section concers implementing SGDL. The main goal is to do bayesian learning from large scale datasets. The main idea is to combine
two type of algorithms:
- *Stochastic Gradient Descent* algorithm (aka. Robbins-Monro) [\ref{SGD}]: which stochastically optimize a likelihood.
- *Langevin dynamics* [\ref{LD}]: which injects noise into the parameter updates in such a way that the trajectory of the parameters will converge to the full posterior distribution rather than just the maximum a posteriori mode.

The resulting algorithm starts off being similar to stochastic optimization, then automatically transitions to one that simulates samples from the posterior using Langevin dynamics.

$$
\tag{1}
\label{SGD}
\Delta\theta_t = \frac{\epsilon_t}{2}\left(\nabla\log p(\theta_t) + \frac{N}{n}\sum_{i=1}^{n}\nabla\log p(x_{ti}|\theta_t)\right)
$$

$$
\tag{2}
\label{LD}
\Delta\theta_t = \frac{\epsilon}{2}\left(\nabla\log p(\theta_t) + \sum_{i=1}^{N}\nabla\log p(x_{i}|\theta_t)\right) + \eta_t, \qquad \eta_t\sim\mathcal{N}(0, \epsilon)
$$

In order to this method to work, i.e. two ensure that the method will converge to a local maximum, two main are required, and they concern the step size:
1. $\sum_{t=1}^{\infty}\epsilon_t = \infty$: to ensure that the parameters will reah the high probability reagions whatever the starting point.
2. $\sum_{t=1}^{\infty}\epsilon_t^2 < \infty$: to ensure that the parameters will converge to the mode and not just bouncing around it.

#### **2.1. Preliminaries**

#### **2.2. Stochastic Gradient Langevin Dynamics (SGLD)**

Based on the similarities between the two algorithms (1) and (3), the idea is to combine the two approaches. This will lead to an efficient use of large data sets while allowing to take into account the uncertainty of the parameters in a Bayesian way.

The process is as follows: 
1. use stochastic gradients descent
2. add an amount of Gaussian noise balanced against the parameter uncertainty.
3. allow the step sizes to go to zero. :

Thus the proposed update is given by:

$$
\tag{3}
\label{SGDL}
\Delta\theta_t = \frac{\epsilon_t}{2}\left(\nabla\log p(\theta_t) + \frac{N}{n}\sum_{i=1}^{n}\nabla\log p(x_{ti}|\theta_t)\right) + \eta_t, \qquad \eta_t\sim\mathcal{N}(0, \epsilon)
$$

#### **2.3. Posterior Sampling**

The idea is that the samples collection should start after the algorithm has entered its posterior sampling phase, which will not happen until after it becomes Langevin dynamics. The main condition to ensure that the algorithm is in its posterior sampling phase is given by:

$$
\tag{4}
\label{cond}
\alpha = \frac{\epsilon_tN^2}{4n}\lambda_{max}(M^{1/2}V_sM^{1/2}) \ll 1
$$
Where $\lambda_{max}$ is the larget eignevalue, $M$ is the preconditioning matrix and $V$ is emprical variance of the paramaters.

### ■ **<a name="eval">3. Evaluation</a>** [(&#8593;)](#content)
In this last section, we evaluate our algorithm implemented above on some classical examples. We start by a mixture of Gaussians, then we appy it on a Bayesian logistic regression model and at last on the Independent Components Analysis model.

#### **3.1. Mixture of Gaussians**
To show that our method works well, we start by applying it on a very basic and simple example with two parameters. This first example is the mixture of Gaussians:
- $\theta_1\sim N(0, \sigma_1^2)$ and $\theta_2\sim N(0, \sigma_2^2)$ where $\sigma_1^2=10$, $\sigma_2^2=1$ 
- $x_i\sim N(\theta_1, \sigma_x^2) + \dfrac12 N(\theta_1+\theta_2, \sigma_x^2)$ where $\sigma_x^2=2$

We draw 100 data points from the model with $\theta_1=0$ and $\theta_2=1$.

#### **3.2. Logistic Regression**
For this second example, we apply stochastic gradient Langevin algorithm to a Bayesian logistic regression model.
We will be using the data from the *UCI* dataset, more specificaly, the *a9a* dataset which consists of 32561 observations and 123 features.

$$
p(y_i|x_i) = \sigma(y_i\beta^Tx_i)
$$
Where $\beta$ are the parameters, and $\sigma$ is the sigmoid function.

#### **3.3. Independent Components Analysis**

### **References** 
@inproceedings{welling2011bayesian,
  title={Bayesian learning via stochastic gradient Langevin dynamics},
  author={Welling, Max and Teh, Yee W},
  booktitle={Proceedings of the 28th international conference on machine learning (ICML-11)},
  pages={681--688},
  year={2011},
  organization={Citeseer}
}

---
<p style="text-align: center;">Copyright © 2021 Omar Ikne & Zakaria Boulkhir</p>